# Initialization

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import copy
import random
from collections import namedtuple, deque
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt

env = UnityEnvironment(file_name='Reacher.app')
# environment brains
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# initial reset
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# show state space
states = env_info.vector_observations
state_size = states.shape[1]
print("state size: ", state_size)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
state size:  33


# Models

## Actor

In [2]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=1000, fc2_units=1000):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()

    def reset_parameters(self):
        """
        sets the weights of the network to a uniforma distribution betweenn -0.003 and 0.003
        """
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return F.tanh(self.fc3(x))


## Critic

In [3]:
class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=1000, fc2_units=1000):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fcs1 = nn.Linear(state_size, fcs1_units)
        self.fc2 = nn.Linear(fcs1_units+action_size, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        """
        sets the weights of the network to a uniforma distribution betweenn -0.003 and 0.003
        """
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.relu(self.fcs1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Memory

In [4]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

# Noise

In [5]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for _ in range(len(x))])
        self.state = x + dx
        return self.state

# Agent

In [18]:
#Constants
LRA = 0.0001
LRC = 0.0001
WEIGHT_DECAY = 0
BUFFER_SIZE = 10000000
BATCH_SIZE=512
UPDATE_SIZE=10
GAMMA=0.99
TAU=0.001
SEED=2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Meta agent that learns from sub-agents interacting with the environment."""
    
    def __init__(self, num_agents=20, state_size=33, action_size=4):
        """Initialize the Agent object.
        
        Params
        ======
            num_agents (int): number of sub-agents
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.num_agents = num_agents
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = BATCH_SIZE
        self.update_size = UPDATE_SIZE
        self.gamma = GAMMA
        self.tau = TAU
        self.seed = random.seed(SEED)

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size, SEED).to(device)
        self.actor_target = Actor(state_size, action_size, SEED).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LRA)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size, action_size, SEED).to(device)
        self.critic_target = Critic(state_size, action_size, SEED).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LRC, weight_decay=WEIGHT_DECAY)

        # Noise process
        self.noise = OUNoise(action_size, SEED)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, SEED)
    
    def save(self, state, action, reward, next_state, done):
        """Save experience in replay memory"""
        self.memory.add(state, action, reward, next_state, done)

    def act(self, state, i_episode, add_noise=True):
        """Returns actions for given state as per current policy.  When adding noise, make sure
           each action gets a different noise sample AND dampen that noise over time.
        """
        state = torch.from_numpy(state).float().to(device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            for a in action:
                a += ((1/math.sqrt(i_episode)) * self.noise.sample())
        return np.clip(action, -1, 1)

    def reset(self):
        self.noise.reset()

    def step(self):
        """Use random sample from buffer to learn, if there are enough samples in the replay buffer.
        Note that we do this multiple times based on the number of agents and the configurable
        update size.
        """
        if len(self.memory) > self.batch_size:
            for _ in range(self.num_agents * self.update_size):
                experiences = self.memory.sample()
                self.learn(experiences, self.gamma)

    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, self.tau)
        self.soft_update(self.actor_local, self.actor_target, self.tau)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

# Train

In [19]:

def ddpg(n_episodes=2000, max_t=1000):
    """
    Main training method for agent with environment loop.
    
    Params
    ======
        n_episodes: max amount of episodes to train for
        max_t: max amount of time steps for each episode
    
    """
    
    scores_deque = deque(maxlen=100)
    scores = []
    update_after_episode=True
    update_t=-1
    print_every = 1
    for i_episode in range(1, 2000+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        agent.reset()
        agent_scores = np.zeros(num_agents)
        for t in range(max_t):
            actions = agent.act(states, i_episode)
            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent_scores += rewards

            # Save the transitions from all agents in the agent's replay buffer.
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.save(state, action, reward, next_state, done)
                         
            states = next_states
            if np.any(dones):
                break
        
        #update at the end of each episode
        agent.step()
    
        #update scores
        score = np.mean(agent_scores)
        scores_deque.append(score)
        scores.append(score) 
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if i_episode % 20 == 0:
            torch.save(agent.actor_local.state_dict(),  'actor.pt')
            torch.save(agent.critic_local.state_dict(), 'critic.pt')
        if np.mean(scores_deque) >= 30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(),  'solved_actor.pt')
            torch.save(agent.critic_local.state_dict(), 'solved_critic.pt')
            
    return scores

In [ ]:
#run and train agent

agent = Agent()
scores = ddpg(agent)

Episode 1	Average Score: 0.01
Episode 2	Average Score: 0.38
Episode 3	Average Score: 0.47
Episode 4	Average Score: 0.56
Episode 5	Average Score: 0.54
Episode 6	Average Score: 0.54
Episode 7	Average Score: 0.55
Episode 8	Average Score: 0.57
Episode 9	Average Score: 0.56
Episode 10	Average Score: 0.59


# Plot Results

In [8]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

NameError: name 'plt' is not defined

# Play Trained Agent

In [ ]:

agent.actor_local.load_state_dict(torch.load('solved_actor.pth'))
agent.critic_local.load_state_dict(torch.load('solved_critic.pth'))

env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations
agent.reset()   
scores = []

for i in range(1000):
    actions = agent.act(states, i_episode)
    env_info = env.step(actions)[brain_name]           # send all actions to the environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    
    agent_scores += rewards
    states = next_states
    if np.any(dones):
        break
    
        
env.close()